In [2]:
import pandas as pd
import numpy as np
import mplfinance as mpf
from pandas_datareader import data as pdr
import re

In [2]:
data = pd.read_csv('tables', sep='\t\t|\t')
data.head()

c:\Users\micha\Projects\StockMarketAnalysis\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,.,代码,名称,..1,涨幅,现价,涨跌,买价,卖价,总手,...,总股本,流通股本,利润总额,净利润增长率,每股净资产,金叉个数,散户数量,自选时间,自选价格,自选收益
0,4,SH600000,浦发银行,--,+1.05%,8.65,+0.09,8.64,8.65,20471995,...,29352166000,29352166000,47348000000亿,-7.17%,18.83,0,0.70,--,--,--
1,4,SH600004,白云机场,--,-1.89%,11.97,-0.23,11.97,11.98,18248019,...,2366718300,2069320500,-558157190亿,-63.72%,7.71,0,27.43,--,--,--
2,4,SH600006,东风汽车,--,+3.26%,6.97,+0.22,6.96,6.97,46696441,...,2000000000,2000000000,510247580亿,4.84%,4.08,9,-21.63,--,--,--
3,4,SH600007,中国国贸,--,+0.70%,14.38,+0.10,14.37,14.38,2080366,...,1007282530,1007282530,1035737310亿,26.65%,8.33,1,0.00,--,--,--
4,4,SH600008,首创环保,--,+1.58%,3.22,+0.05,3.21,3.22,104372285,...,7340590700,7340590700,2533266300亿,106.2%,2.37,2,2.50,--,--,--


In [64]:
import re
import yfinance as yf
subset = data.loc[data['所属行业'] == '银行']
subset.head()

exDict = {
    'SH': 'SS'
}

bank = subset.loc[subset['    名称'] == '招商银行']

input = bank['代码'].values[0]
digits = re.findall('[0-9]+', input)
if digits:
    exchange = input.replace(digits[0], '')
    ticker = f'{digits[0]}.{exDict[exchange]}'
    print(ticker)
    stock = yf.Ticker(ticker)
else:
    print('nothing found')


600036.SS


In [13]:
def correct_ticker(input):
    exDict = {
        'SH': 'SS',
        'SZ': 'SZ'
    }
    if type(input) == str:
        try:
            digits = re.findall('[0-9]+', input)
            if digits:
                exchange = input.replace(digits[0], '')
                ticker = f'{digits[0]}.{exDict[exchange]}'
                return ticker
        except:
            print(input)
    return None

def clean_data(data):
    data['Name'] = data.loc[data['    名称'] != '--']['    名称'].append(data.loc[data['    名称'] == '--']['代码'])
    tickers = data.loc[data['    名称'] != '--']['代码'].append(data.loc[data['    名称'] == '--']['.'])
    # tickers.head()
    data['Ticker'] = tickers.apply(lambda row : correct_ticker(row))
    data['Industry'] = data.loc[data['    名称'] != '--']['所属行业'].append(data.loc[data['    名称'] == '--']['细分行业'])

    return data

def load_txt(path):
    df = pd.read_csv(path, sep='\t+')
    df['代码'] = df['代码'].apply(lambda row : correct_ticker(row))
    return df
# idx = df.index[df['.'] != '4']
# lCols = np.delete(df.columns.values, np.where(df.columns.values=='自选收益')[0][0])
# rCols = np.delete(df.columns.values, np.where(df.columns.values=='.')[0][0])
# df.loc[idx, rCols] = df.loc[idx, lCols].values
# # df['代码'] = df['代码'].apply(lambda row : correct_ticker(row))
# df[df['代码'] == '64']
# clean_data(df)
df = load_txt('tables2')
# df.columns
print(df['所属行业'].unique())
df[df['所属行业'] == '--']

['银行' '机场航运' '汽车整车' '房地产开发' '环保' '钢铁' '电力' '公路铁路运输' '港口航运' '石油加工贸易' '证券'
 '专用设备' '传媒' '国防军工' '建筑装饰' '通信服务' '综合' '保险及其他' '景点及旅游' '医疗器械' '医药商业' '物流'
 '贸易' '饮料制造' '黑色家电' '化学制药' '化工合成材料' '光学光电子' '食品加工制造' '化学原料' '家用轻工' '中药'
 '汽车零部件' '电力设备' '化学制品' '养殖业' '非汽车交运' '计算机设备' '造纸' '通信设备' '服装家纺' '种植业与林业'
 '工业金属' '小金属' '零售' '通用设备' '煤炭开采加工' '农产品加工' '消费电子' '计算机应用' '其他社会服务' '纺织制造'
 '生物制品' '半导体及元件' '建筑材料' '农业服务' '包装印刷' None '美容护理' '金属新材料' '油气开采及服务'
 '酒店及餐饮' '其他电子' '汽车服务' '燃气' '白色家电' '教育' '贵金属' '互联网电商' '专业服务' '医疗服务' '--'
 '保险' '电子化学品' '自动化设备' '仪器仪表' '小家电' '厨卫电器' '房地产服务' '非金属材料']


c:\Users\micha\Projects\StockMarketAnalysis\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,代码,名称,总市值,流通市值,净利润?,市净率,每股盈利,细分行业,所属行业
794,601206.SS,海尔施,--,--,77045083,0.0,0.370,--,--
1119,603302.SS,鑫广绿环,--,--,185000000,0.0,0.350,--,--
1907,688688.SS,蚂蚁集团,--,--,21234497000,0.0,0.800,--,--
4133,300728.SZ,天常股份,--,--,51370000,0.0,0.353,--,--
